In [1]:
# import all dependencies
import numpy as np
import pandas as pd
import copy
import yfinance as yf
import datetime as dt
from plotly.offline import plot
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# atr function to detect breached price to initatiate a stop loss
def atr(df, n):
    df = df.copy()
    df['high-low'] = abs(df['High'] - df['Low'])
    df['high-pc'] = abs(df['High'] - df['Adj Close'].shift(1))
    df['low-pc'] = abs(df['Low'] - df['Adj Close'].shift(1))
    df['true-range'] = df[['high-low', 'high-pc', 'low-pc']].max(axis=1, skipna=False)
    df['atr'] = df['true-range'].rolling(n).mean()
    df.dropna(inplace=True)
    return df['atr']

In [3]:
# kpis
def cagr(df):
    df = df.copy()
    df['cumulative-return'] = (1 + df['return']).cumprod()
    n = len(df) / (252) # number of trading days per 1 year
    cagr = (df['cumulative-return'].tolist()[-1])**(1/n) - 1
    return cagr

def volatility(df):
    df = df.copy()
    annual_volatility = df['return'].std() * np.sqrt(252)
    return annual_volatility

def sharpe(df, rf):
    df = df.copy()
    sr = (cagr(df) - rf) / volatility(df)
    return sr

def maximum_drawdown(df):
    df = df.copy()
    df['cumulative-return'] = (1 + df['return']).cumprod() # value today
    df['cumulative-rolling-max'] = df['cumulative-return'].cummax()
    df['drawdown'] = df['cumulative-rolling-max'] - df['cumulative-return']
    df['drawdown-percent'] = df['drawdown'] / df['cumulative-rolling-max']
    m_dd = df['drawdown-percent'].max()
    return m_dd

def calmar_ratio(df):
    df = df.copy()
    c_r = cagr(df) / maximum_drawdown(df)
    return c_r

In [4]:
# time horizon set-up
start = dt.datetime.today() - dt.timedelta(365)
end = dt.datetime.today()

In [5]:
# NASDAQ most active by dollars stocks
# original_tickers = ['AMZN', 'MSFT', 'NVDA', 'NFLX', 'ADBE']
original_tickers = ['TSLA', 'AAPL', 'AMZN', 'MSFT', 'QCOM', 'FB', 'AMD', 'NVDA', 'NFLX', 'ADBE']
original_ohlcvs = {}

In [6]:
# downloading data for all tickers
for ticker in original_tickers:
    original_ohlcvs[ticker] = yf.download(ticker, start, end, interval='1d')

# assigning new tickers
tickers = original_ohlcvs.keys()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [7]:
# basic data structure to implement back-testing
ohlcvs = copy.deepcopy(original_ohlcvs)
tickers_signal = {} # { 'MSFT': 'buy', 'AAPL': '', 'INTC': 'sell' }
tickers_signal_track = {}
tickers_return = {}

In [8]:
# calculate technical indicators
for ticker in tickers:
    print(f'Calculating ATR and rolling max price for {ticker}')
    ohlcvs[ticker]['atr'] = atr(ohlcvs[ticker], 20)
    ohlcvs[ticker]['rolling-max-price'] = ohlcvs[ticker]['High'].rolling(20).max()
    ohlcvs[ticker]['rolling-min-price'] = ohlcvs[ticker]['Low'].rolling(20).max()
    ohlcvs[ticker]['rolling-max-volume'] = ohlcvs[ticker]['Volume'].rolling(20).max()
    ohlcvs[ticker].dropna(inplace=True)
    tickers_signal[ticker] = ''
    tickers_signal_track[ticker] = []
    tickers_return[ticker] = []

Calculating ATR and rolling max price for TSLA
Calculating ATR and rolling max price for AAPL
Calculating ATR and rolling max price for AMZN
Calculating ATR and rolling max price for MSFT
Calculating ATR and rolling max price for QCOM
Calculating ATR and rolling max price for FB
Calculating ATR and rolling max price for AMD
Calculating ATR and rolling max price for NVDA
Calculating ATR and rolling max price for NFLX
Calculating ATR and rolling max price for ADBE


In [9]:
ohlcvs['AMZN']

,Open,High,Low,Close,Adj Close,Volume,atr,rolling-max-price,rolling-min-price,rolling-max-volume
Date,,,,,,,,,,
2020-01-15,1872.250000,1878.859985,1855.089966,1862.020020,1862.020020,2896600,28.794995,1917.819946,1895.800049,6186600.0
2020-01-16,1882.989990,1885.589966,1866.020020,1877.939941,1877.939941,2659500,28.833990,1917.819946,1895.800049,6186600.0
2020-01-17,1885.890015,1886.640015,1857.250000,1864.719971,1864.719971,3997300,29.511493,1917.819946,1895.800049,6186600.0
2020-01-21,1865.000000,1894.270020,1860.000000,1892.000000,1892.000000,3707800,30.278497,1917.819946,1895.800049,6186600.0
2020-01-22,1896.089966,1902.500000,1883.339966,1887.459961,1887.459961,3216300,30.210498,1917.819946,1895.800049,6186600.0
...,...,...,...,...,...,...,...,...,...,...
2020-12-07,3156.479980,3180.760010,3141.689941,3158.000000,3158.000000,2751300,71.418457,3289.000000,3190.050049,7190400.0
2020-12-08,3158.899902,3184.129883,3120.020020,3177.290039,3177.290039,3286300,64.660950,3248.949951,3190.050049,6591000.0
2020-12-09,3167.889893,3174.429932,3088.000000,3104.199951,3104.199951,4100800,62.912451,3248.949951,3190.050049,4708900.0


In [10]:
# identifying signals and calculating return and incorporate stop-loss accordingly
for ticker in tickers:
    print(f'Calculating returns for {ticker}')
    for i in range(len(ohlcvs[ticker])):
        
        if (tickers_signal[ticker] == ''):
            tickers_return[ticker].append(0) # no signal equals no return for that candle     
            
            if (ohlcvs[ticker]['High'][i] >= ohlcvs[ticker]['rolling-max-price'][i] and 
                ohlcvs[ticker]['Volume'][i] > 1.2 * ohlcvs[ticker]['rolling-max-volume'][i-1]): # rule for long break-out
                tickers_signal[ticker] = 'Buy' # initiate a buy signal
                tickers_signal_track[ticker].append('Start Buy')
            elif (ohlcvs[ticker]['Low'][i] <= ohlcvs[ticker]['rolling-min-price'][i] and 
                ohlcvs[ticker]['Volume'][i] > 1.2 * ohlcvs[ticker]['rolling-max-volume'][i-1]): # rule for short break-out
                tickers_signal[ticker] = 'Sell' # initiate a sell signal
                tickers_signal_track[ticker].append('Start Sell')
            else:
                tickers_signal_track[ticker].append('dnt')
                
        elif (tickers_signal[ticker] == 'Buy'):        
            if (ohlcvs[ticker]['Low'][i] < ohlcvs[ticker]['Close'][i-1] - ohlcvs[ticker]['atr'][i-1]): # low price breached, stop loss
                tickers_signal[ticker] = '' # change from buy signal to nth to close off
                tickers_signal_track[ticker].append('Close Buy')
                tickers_return[ticker].append(((ohlcvs[ticker]['Close'][i-1] - ohlcvs[ticker]['atr'][i-1]) / ohlcvs[ticker]['Close'][i-1]) - 1)
            else:
                tickers_return[ticker].append((ohlcvs[ticker]['Close'][i] / ohlcvs[ticker]['Close'][i-1]) - 1)
                tickers_signal_track[ticker].append('Continue')
                
        elif (tickers_signal[ticker] == 'Sell'):
            if (ohlcvs[ticker]['High'][i] > ohlcvs[ticker]['Close'][i-1] + ohlcvs[ticker]['atr'][i-1]): # high price breached, stop loss
                tickers_signal[ticker] = '' # change from sell signal to nth to close off
                tickers_signal_track[ticker].append('Close Sell')
                tickers_return[ticker].append(((ohlcvs[ticker]['Close'][i-1] / (ohlcvs[ticker]['Close'][i-1] + ohlcvs[ticker]['atr'][i-1]))) - 1)
            else:
                tickers_return[ticker].append((ohlcvs[ticker]['Close'][i-1] / ohlcvs[ticker]['Close'][i]) - 1)
                tickers_signal_track[ticker].append('Continue')
    
    ohlcvs[ticker]['return'] = np.array(tickers_return[ticker])
    ohlcvs[ticker]['signal'] = np.array(tickers_signal_track[ticker])

Calculating returns for TSLA
Calculating returns for AAPL
Calculating returns for AMZN
Calculating returns for MSFT
Calculating returns for QCOM
Calculating returns for FB
Calculating returns for AMD
Calculating returns for NVDA
Calculating returns for NFLX
Calculating returns for ADBE


In [19]:
# create figure
signal_ticker = 'NVDA'

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=ohlcvs[signal_ticker].index, y=ohlcvs[signal_ticker]['Adj Close'], name='Price'))

for i in range(len(ohlcvs[signal_ticker])):
    if (ohlcvs[signal_ticker]['signal'][i] == 'Start Buy'):
        fig.add_trace(go.Scatter(x=[ohlcvs[signal_ticker].index[i], ohlcvs[signal_ticker].index[i]], y=[0,1], name='Start Buy', marker_color='rgba(0, 255, 0, 1)'), secondary_y=True)
    if (ohlcvs[signal_ticker]['signal'][i] == 'Close Buy'):
        fig.add_trace(go.Scatter(x=[ohlcvs[signal_ticker].index[i], ohlcvs[signal_ticker].index[i]], y=[0,1], name='Close Buy', marker_color='rgba(0, 255, 0, 0.1)'), secondary_y=True)
    if (ohlcvs[signal_ticker]['signal'][i] == 'Start Sell'):
        fig.add_trace(go.Scatter(x=[ohlcvs[signal_ticker].index[i], ohlcvs[signal_ticker].index[i]], y=[0,1], name='Start Sell', marker_color='rgba(255, 0, 0, 1)'), secondary_y=True)
    if (ohlcvs[signal_ticker]['signal'][i] == 'Close Sell'):
        fig.add_trace(go.Scatter(x=[ohlcvs[signal_ticker].index[i], ohlcvs[signal_ticker].index[i]], y=[0,1], name='Close Sell', marker_color='rgba(255, 0, 0, 0.1)'), secondary_y=True)

# format figure
fig.update_layout(title_text=f'S1 Signals Backtested {dt.datetime.today()} based on Price of {signal_ticker}')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Price')
fig.update_yaxes(title_text='Reference', secondary_y=True)

In [21]:
# create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=ohlcvs[signal_ticker].index, y=ohlcvs[signal_ticker]['Volume'], name='Volume'))

for i in range(len(ohlcvs[signal_ticker])):
    if (ohlcvs[signal_ticker]['signal'][i] == 'Start Buy'):
        fig.add_trace(go.Scatter(x=[ohlcvs[signal_ticker].index[i], ohlcvs[signal_ticker].index[i]], y=[0,1], name='Start Buy', marker_color='rgba(0, 255, 0, 1)'), secondary_y=True)
    if (ohlcvs[signal_ticker]['signal'][i] == 'Close Buy'):
        fig.add_trace(go.Scatter(x=[ohlcvs[signal_ticker].index[i], ohlcvs[signal_ticker].index[i]], y=[0,1], name='Close Buy', marker_color='rgba(0, 255, 0, 0.1)'), secondary_y=True)
    if (ohlcvs[signal_ticker]['signal'][i] == 'Start Sell'):
        fig.add_trace(go.Scatter(x=[ohlcvs[signal_ticker].index[i], ohlcvs[signal_ticker].index[i]], y=[0,1], name='Start Sell', marker_color='rgba(255, 0, 0, 1)'), secondary_y=True)
    if (ohlcvs[signal_ticker]['signal'][i] == 'Close Sell'):
        fig.add_trace(go.Scatter(x=[ohlcvs[signal_ticker].index[i], ohlcvs[signal_ticker].index[i]], y=[0,1], name='Close Sell', marker_color='rgba(255, 0, 0, 0.1)'), secondary_y=True)

# format figure
fig.update_layout(title_text=f'S1 Signals Backtested {dt.datetime.today()} based on Volume of {signal_ticker}')
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Volume')
fig.update_yaxes(title_text='Reference', secondary_y=True)

In [13]:
# calculating overall strategy's KPIs
strategy = pd.DataFrame()
for ticker in tickers:
    strategy[ticker] = ohlcvs[ticker]['return']
strategy['return'] = strategy.mean(axis=1) # assume equal capital allocation
strategy['cumulative-return'] = (1 + strategy['return']).cumprod()
strategy_cagr = cagr(strategy)
strategy_volatility = volatility(strategy)
strategy_sharpe = sharpe(strategy, 0.01)
strategy_max_dd = maximum_drawdown(strategy)
strategy_calmar_ratio = calmar_ratio(strategy)

In [14]:
strategy

,TSLA,AAPL,AMZN,MSFT,QCOM,FB,AMD,NVDA,NFLX,ADBE,return,cumulative-return
Date,,,,,,,,,,,,
2020-01-15,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.000000
2020-01-16,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.000000
2020-01-17,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,1.000000
2020-01-21,0.0,0.0,0.0,0.0,-0.014284,0.0,0.000000,0.0,0.000000,0.0,-0.001428,0.998572
2020-01-22,0.0,0.0,0.0,0.0,-0.016607,0.0,0.000000,0.0,0.037147,0.0,0.002054,1.000623
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-07,0.0,0.0,0.0,0.0,0.002538,0.0,0.000319,0.0,0.000000,0.0,0.000286,1.053716
2020-12-08,0.0,0.0,0.0,0.0,0.004936,0.0,-0.012225,0.0,0.000000,0.0,-0.000729,1.052948
2020-12-09,0.0,0.0,0.0,0.0,-0.017506,0.0,-0.033023,0.0,0.000000,0.0,-0.005053,1.047627


In [15]:
print(f'Strategy CAGR: {strategy_cagr}')
print(f'Strategy Volatility: {strategy_volatility}')
print(f'Strategy Sharpe: {strategy_sharpe}')
print(f'Strategy Maximum Drawdown: {strategy_max_dd}')
print(f'Strategy Calmar Ratio: {strategy_calmar_ratio}')

Strategy CAGR: 0.048688933703993964
Strategy Volatility: 0.058434945792265346
Strategy Sharpe: 0.6620855582125819
Strategy Maximum Drawdown: 0.030169190191737857
Strategy Calmar Ratio: 1.613862798257274


In [16]:
# KPIs for each stocks
individual_cagr = {}
individual_volatility = {}
individual_sharpe = {}
individual_max_dd = {}
individual_calmar_ratio = {}

for ticker in tickers:
    print(f'Calculating KPIs for {ticker}')
    individual_cagr[ticker] = cagr(ohlcvs[ticker])
    individual_volatility[ticker] = volatility(ohlcvs[ticker])
    individual_sharpe[ticker] = sharpe(ohlcvs[ticker], 0.01)
    individual_max_dd[ticker] = maximum_drawdown(ohlcvs[ticker])
    individual_calmar_ratio[ticker] = calmar_ratio(ohlcvs[ticker])

Calculating KPIs for TSLA
Calculating KPIs for AAPL
Calculating KPIs for AMZN
Calculating KPIs for MSFT
Calculating KPIs for QCOM
Calculating KPIs for FB
Calculating KPIs for AMD
Calculating KPIs for NVDA
Calculating KPIs for NFLX
Calculating KPIs for ADBE


In [17]:
individual_cagr

{'TSLA': 0.3880325364546193,
 'AAPL': -0.037061411583023784,
 'AMZN': 0.022333585091909303,
 'MSFT': 0.01635672573255431,
 'QCOM': -0.1507393163187971,
 'FB': -0.12906622290988345,
 'AMD': -0.02839615234193693,
 'NVDA': 0.2713393750041604,
 'NFLX': 0.017248601229448157,
 'ADBE': 0.08635421346366923}

In [18]:
# create figure
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=strategy.index, y=strategy['cumulative-return'], name='Cumulative Return'))

# format figure
fig.update_layout(title_text=f'Cumulative Return of Portfolio of S1 - Resistance Breakout Backtested {dt.datetime.today()}')
fig.update_xaxes(title_text='Period')
fig.update_yaxes(title_text='Return')